# Install Dependency

In [5]:
!pip install transformers
!pip install tqdm
!pip install matplotlib

  Using cached matplotlib-3.7.1-cp39-cp39-win_amd64.whl (7.6 MB)
  Using cached contourpy-1.0.7-cp39-cp39-win_amd64.whl (160 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.39.4-py3-none-any.whl (1.0 MB)
  Using cached kiwisolver-1.4.4-cp39-cp39-win_amd64.whl (55 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)


# Import lib

In [6]:
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, pipeline
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

In [7]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to C:\Users\Imad Eddine
[nltk_data]     Hajjane\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Imad Eddine
[nltk_data]     Hajjane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Imad Eddine
[nltk_data]     Hajjane\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Imad Eddine
[nltk_data]     Hajjane\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [9]:
pd.set_option('display.max_colwidth', None)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
TRANSFORMER = "distilbert-base-uncased"
MODEL_PATH = os.path.join("Transformer", "Model", "Model")

# Import data and clean it

In [12]:
stop_words = set(stopwords.words('english')).difference(set(("never", "not","no")))
def removeSpeCara(s:str):
    return re.sub(r"[^a-zA-Z]", "", s) 

#retirer les url
def remove_url(s:str)->str:
    url_pattern = re.compile(r"http?://\S+|https?://\S+|www\.\S+|//S+")
    return url_pattern.sub("r", s)

#retirer les html 
def remove_html(s:str)->str:
    html_pattern = re.compile(r"<.*?>")
    return html_pattern.sub("r", s)

# retirer les emojies
def remove_emoji(s:str)->str:
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
                      "]+", re.UNICODE)
    return emoji_pattern.sub("r", s)

def clean_and_lemmatize_string(s:str):
    l = []
    s_ = " ".join([remove_html(remove_url(word)) for word in s.split()])
    for word in word_tokenize(s_):
        word_ = removeSpeCara((remove_emoji(word)))
        if not word_ in stop_words and len(word_) > 1:
                l.append(word_.lower())
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    lemma_function = WordNetLemmatizer()
    return " ".join([lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(l)])

In [13]:
dataframe = pd.read_csv(os.path.join("..", "tweets_01-08-2021.csv"))[["id", "text"]]
dataframe_res = dataframe.copy()
dataframe_res["text_clean"] = dataframe_res["text"]

In [14]:
tqdm.pandas()

dataframe_res["text_clean"] = dataframe_res["text_clean"].progress_apply(lambda s : clean_and_lemmatize_string(s))
dataframe_res = dataframe_res[dataframe_res["text_clean"].str.len() >= 3]
X = dataframe_res["text_clean"].tolist()

100%|██████████| 56571/56571 [00:44<00:00, 1267.69it/s]


In [15]:
dataframe_res

,id,text,text_clean
0,98454970654916608,Republicans and Democrats have both created our economic problems.,republican democrat create economic problem
1,1234653427789070336,"I was thrilled to be back in the Great city of Charlotte, North Carolina with thousands of hardworking American Patriots who love our Country, cherish our values, respect our laws, and always put AMERICA FIRST! Thank you for a wonderful evening!! #KAG2020 https://t.co/dNJZfRsl9y",thrill back great city charlotte north carolina thousand hardworking american patriot love country cherish value respect law always put america first thank wonderful evening kag
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance court obtained by CBS News questions where there will be further disciplinary action and cho…,rt cbsherridge read letter surveillance court obtain cbs news question disciplinary action cho
3,1304875170860015617,"The Unsolicited Mail In Ballot Scam is a major threat to our Democracy, &amp; the Democrats know it. Almost all recent elections using this system, even though much smaller &amp; with far fewer Ballots to count, have ended up being a disaster. Large numbers of missing Ballots &amp; Fraud!",the unsolicited mail in ballot scam major threat democracy amp democrat know almost recent election use system even though much small amp far few ballots count end disaster large number miss ballot amp fraud
4,1218159531554897920,RT @MZHemingway: Very friendly telling of events here about Comey's apparent leaking to compliant media. If you read those articles and tho…,rt mzhemingway very friendly tell event comey apparent leaking compliant medium if read article tho
...,...,...,...
56566,1319485303363571714,RT @RandPaul: I don’t know why @JoeBiden thinks he can continue to lie about this. \n\nHe wants to ban fracking and end all fossil fuels like…,rt randpaul know joebiden think continue lie he want ban fracking end fossil fuel like
56567,1319484210101379072,RT @EliseStefanik: President @realDonaldTrump excels at communicating directly to the American people. \n\nJoe Biden communicates to the DC B…,rt elisestefanik president realdonaldtrump excels communicate directly american people joe biden communicates dc
56568,1319444420861829121,RT @TeamTrump: LIVE: Presidential Debate #Debates2020\n\nText VOTE to 88022 https://t.co/UeQOquVxR2,rt teamtrump live presidential debate debate text vote
56569,1319384118849949702,Just signed an order to support the workers of Delphi Corporation and make sure that we protect the pensions of all American workers! Obama-Biden FAILED American workers and FAILED the workers of Delphi. I ALWAYS put American workers FIRST!,just sign order support worker delphi corporation make sure protect pension american worker obamabiden fail american worker fail worker delphi always put american worker first


# Prediction using Bert model

In [16]:
class twitterDataset(torch.utils.data.Dataset):
    def __init__(self, encoding, size):
        self.encoding = encoding
        self.size = size
    def __getitem__(self, idx):
        item = {key : torch.tensor(val[idx]) for key, val in self.encoding.items()}
        return item
    
    def __len__(self):
        return self.size

In [17]:
tokenizer = DistilBertTokenizerFast.from_pretrained(TRANSFORMER)
dataset2predict = twitterDataset(tokenizer(X, padding="max_length", truncation=True), len(X))

In [18]:
gc.collect()
torch.cuda.empty_cache()

In [19]:
model_bert = DistilBertForSequenceClassification.from_pretrained(MODEL_PATH).to(device)

In [20]:
def predict_(model, encoding, device):
    input_ids = encoding["input_ids"].detach().clone().to(device)
    att_mask = encoding["attention_mask"].detach().clone().to(device)
    with torch.no_grad():
        y = np.argmax(model(input_ids.unsqueeze(0), att_mask.unsqueeze(0))[0].to('cpu').numpy())
    return y

In [21]:
dataframe_res["Opinion_bert"] = [("Positive" if predict_(model_bert, sentence, device) else "Negative") for sentence in tqdm(dataset2predict)]


100%|██████████| 55242/55242 [06:22<00:00, 144.58it/s]


In [22]:
dataframe_res = dataframe_res.drop("text_clean", axis=1)

In [23]:
dataframe_res

,id,text,Opinion_bert
0,98454970654916608,Republicans and Democrats have both created our economic problems.,Negative
1,1234653427789070336,"I was thrilled to be back in the Great city of Charlotte, North Carolina with thousands of hardworking American Patriots who love our Country, cherish our values, respect our laws, and always put AMERICA FIRST! Thank you for a wonderful evening!! #KAG2020 https://t.co/dNJZfRsl9y",Positive
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance court obtained by CBS News questions where there will be further disciplinary action and cho…,Negative
3,1304875170860015617,"The Unsolicited Mail In Ballot Scam is a major threat to our Democracy, &amp; the Democrats know it. Almost all recent elections using this system, even though much smaller &amp; with far fewer Ballots to count, have ended up being a disaster. Large numbers of missing Ballots &amp; Fraud!",Negative
4,1218159531554897920,RT @MZHemingway: Very friendly telling of events here about Comey's apparent leaking to compliant media. If you read those articles and tho…,Negative
...,...,...,...
56566,1319485303363571714,RT @RandPaul: I don’t know why @JoeBiden thinks he can continue to lie about this. \n\nHe wants to ban fracking and end all fossil fuels like…,Negative
56567,1319484210101379072,RT @EliseStefanik: President @realDonaldTrump excels at communicating directly to the American people. \n\nJoe Biden communicates to the DC B…,Positive
56568,1319444420861829121,RT @TeamTrump: LIVE: Presidential Debate #Debates2020\n\nText VOTE to 88022 https://t.co/UeQOquVxR2,Positive
56569,1319384118849949702,Just signed an order to support the workers of Delphi Corporation and make sure that we protect the pensions of all American workers! Obama-Biden FAILED American workers and FAILED the workers of Delphi. I ALWAYS put American workers FIRST!,Negative


# Save the dataframe

In [24]:
dataframe_res.to_csv(os.path.join("result","trump_tweet_opinion_bert.csv")) 